<a href="https://colab.research.google.com/github/shayanthrn/Acoustic_scene_classification/blob/main/Acoustic_scene_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Acoustic scene classification

Dataset: IEEE AASP CASA Challenge, Available on: http://dcase.community/challenge2013/task-acoustic-scene-classification
<br/>Currently dataset is in my googe drive

In [ ]:
!pip install torchaudio

##import libraries

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
import os
from torchvision.transforms import ToTensor
import numpy as np

##global variables and hyperparameters

In [ ]:
class_map = ["bus","busystreet","office","openairmarket","park","quietstreet","restaurant","supermarket","tube","tubestation"]
BATCH_SIZE = 32

##create dataset class for DCASE2013 dataset

In [12]:
class DCASE13(Dataset):

  def __init__(self,path,class_map):
    super().__init__()
    self.dataset_path = path
    self.class_map = class_map
    self.sample_rate = 16000
    self.mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=self.sample_rate,n_fft=1024,hop_length=512,n_mels=64)
  
  def __len__(self):
    return len(os.listdir(self.dataset_path))
  
  def __getitem__(self,index):
    file_name = os.listdir(self.dataset_path)[index]
    class_name = file_name.split('0')[0].split('1')[0]
    label = class_map.index(class_name)
    file_path = os.path.join(self.dataset_path,file_name)
    signal, sample_rate = torchaudio.load(filepath=file_path)
    #resample if necessary
    if(sample_rate != self.sample_rate):
      resampler = torchaudio.transforms.Resample(sample_rate,self.sample_rate)
      signal = resampler(signal)
    # stereo to mono convert
    if(signal.shape[0]>1):
      signal = torch.mean(signal, dim=0, keepdim=True) 
    signal = self.mel_spectrogram(signal)
    print(signal.shape)
    return signal,label




In [13]:
dcase13 = DCASE13("/content/drive/MyDrive/IEEE_AASP_CASA_Challenge/DCASE13_train",class_map)

In [10]:
dataloader = DataLoader(dcase13 , batch_size=BATCH_SIZE , shuffle=True)